# Requirements

Since this notebook uses a Julia package that is not in the standard library, an environment was created in which it is installed.  Using the `Pkg` module, that environment, `struct_arrrays`, can be activated, so that the `StructArrays` module can be loaded.

In [1]:
using Pkg
Pkg.activate("struct_arrays")
using StructArrays
using BenchmarkTools

# Array of structures

Consider a structure that represents a particle.  A point has an $x$ and $y$ coordinate, and a mass.

In [2]:
struct Particle
    x::Float64
    y::Float64
    mass::Float64
    v_x::Float64
    v_y::Float64
    charge::Int32
end

We create many such particles and store them in a `Vector`.

In [3]:
nr_particles = 10_000_000
particles = Array{Particle}(undef, nr_particles)
for i in eachindex(particles)
    particles[i] = Particle(randn(), randn(), rand(), randn(), randn(), 1)
end

In [4]:
typeof(particles)

Vector{Particle} (alias for Array{Particle, 1})

We want to compute the center of mass and define a function to compute it.

In [5]:
function center_of_mass(particles::Array{Particle})
    x_coord = sum(map(p -> p.x, particles))
    y_coord = sum(map(p -> p.y, particles))
    mass = sum(map(p -> p.mass, particles))
    return x_coord/mass, y_coord/mass
end

center_of_mass (generic function with 1 method)

In [14]:
@benchmark center_of_mass(particles)

BechmarkTools.Trial: 37 samples with 1 evaluations.
 Range (min … max):  129.019 ms … 147.559 ms  ┊ GC (min … max): 0.00% … 11.24%
 Time  (median):     131.979 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   136.593 ms ±   7.411 ms  ┊ GC (mean ± σ):  4.46% ±  4.96%

  ▂█  ▅                                                          
  ██▅███▅▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▅▁█▅▅▅▅▁▅█▁▁▁▁▅██ ▁
  129 ms           Histogram: frequency by time          148 ms <

 Memory estimate: 228.88 MiB, allocs estimate: 7.

The memory access pattern in the previous implementation is far from optimal, for the current data stucture it can be improved by iterating over the particles and computing the $x$ and $y$ coordinates and the total mass one particle at th4e time.

In [7]:
function center_of_mass_iter(particles::Array{Particle})
    x_coord, y_coord, mass = 0.0, 0.0, 0.0
    for particle in particles
        x_coord += particle.x
        y_coord += particle.y
        mass += particle.mass
    end
    return x_coord/mass, y_coord/mass
end

center_of_mass_iter (generic function with 1 method)

In [8]:
@benchmark center_of_mass_iter(particles)

BechmarkTools.Trial: 205 samples with 1 evaluations.
 Range (min … max):  23.991 ms …  26.361 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     24.313 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.360 ms ± 314.726 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▇▃█▁▄▁▄▂ ▁▄▅▇ ▅ ▄▁                                          
  ▄▆████████▅████▆█████▆▄▆▁▅▄▃▅▄▁▁▃▃▃▁▁▁▁▁▁▁▁▃▁▃▃▁▁▁▁▃▁▃▃▁▁▁▁▃ ▄
  24 ms           Histogram: frequency by time         25.5 ms <

 Memory estimate: 32 bytes, allocs estimate: 1.

# Structure of arrays

However, note that we are accessing the `x`, `y` and `mass` field of each particle one after the other.  Storing these fields in arrays would improve the memory access pattern.

In [9]:
particles_sa = StructArray(particles);

In [10]:
typeof(particles_sa)

StructVector{Particle, NamedTuple{(:x, :y, :mass, :v_x, :v_y, :charge), Tuple{Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Int32}}}, Int64} (alias for StructArray{Particle, 1, NamedTuple{(:x, :y, :mass, :v_x, :v_y, :charge), Tuple{Array{Float64, 1}, Array{Float64, 1}, Array{Float64, 1}, Array{Float64, 1}, Array{Float64, 1}, Array{Int32, 1}}}, Int64})

In [11]:
function center_of_mass(particles::StructArray{Particle})
    x_coord = sum(particles.x)
    y_coord = sum(particles.y)
    mass = sum(particles.mass)
    return x_coord/mass, y_coord/mass
end

center_of_mass (generic function with 2 methods)

In [12]:
@benchmark center_of_mass(particles_sa)

BechmarkTools.Trial: 402 samples with 1 evaluations.
 Range (min … max):  12.148 ms …  13.448 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.394 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.414 ms ± 141.881 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▁▄ ▁▄ ▂▄█▇▇▇▆▂                                       
  ▃▁▁▂▂▃▃▄▇██████████████▆▆▆▆▅▃▅▄▃▂▁▃▄▃▂▃▁▂▂▂▂▁▃▂▃▂▁▁▁▃▁▁▁▁▁▁▂ ▄
  12.1 ms         Histogram: frequency by time           13 ms <

 Memory estimate: 32 bytes, allocs estimate: 1.

This data structure is much more suited for the memory access pattern in our original implementation.  All $x$ coordinates are stored continguously in memory, and similar for $y$ and the masses.